In [1]:
import torch
# model_silero_te, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
#                                                                   model='silero_te')
import re
import string
from nltk.tokenize import word_tokenize,TreebankWordTokenizer


In [2]:
import soundfile
# load model and processor
import os
# import librosa
import jiwer
import string
import csv
from autocorrect import Speller
spell = Speller()
from transformers import pipeline

# fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")


def get_gold_hyp_lists(gold_path,test_path,septype):
    def create_hyp_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, 'r') as file:
            lines = file.readlines()
            for idx,line in enumerate(lines):
                line = line.strip()
                if line:
                    sentence_id, sentence = line.split(septype,1)
                    sentence_id=sentence_id+".wav" # for AMI
                    sentence_dict[sentence_id] = sentence
        return sentence_dict

    def create_gold_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, 'r') as file:
            for line in file:
                # parts = line.split(' ', 1) # Libri
                parts = line.split(',', 1) # AMI
                sentence_id, sentence = parts
                if sentence.strip()!='':
                    sentence_dict[sentence_id] = sentence.strip()
        return sentence_dict
    
    def read_csv_to_dict(file_path):
        result_dict = {}
        with open(file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
                key, value = row[0], row[1]
                key=key.split('/')[1]
                # print(key,value)
                # break
                result_dict[key] = value
        return result_dict
    
    gold_dict = create_gold_sentence_dictionary(gold_path)
    # gold_dict = read_csv_to_dict(gold_path)
    test_dict = create_hyp_sentence_dictionary(test_path)
    
    gold_list = []
    test_list = []
    for test_key, test_value in test_dict.items():
        if test_key in gold_dict:
            gold = gold_dict[test_key]
            gold_list.append(gold)
            test_list.append(test_value)
    print(len(test_list))
    return gold_list,test_list

def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ''
    for char in text:
        if char in punctuation_marks:
            modified_text += ' ' + char
        else:
            modified_text += char

    return modified_text

def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ''
    for char in text:
        if char in punctuation_marks:
            modified_text += ' ' + char
        else:
            modified_text += char

    return modified_text

def add_space(reference_list, hypothesis_list):
    reference = []
    hypothesis = []
    for ref, hyp in zip(reference_list, hypothesis_list):
        # Preprocess sentences to keep punctuation as separate words
        reference.append(add_space_before_punctuation(ref))
        hypothesis.append(add_space_before_punctuation(hyp))
        # print(reference,hypothesis)
        # Calculate WER
       
    #     wer = jiwer.wer(reference, hypothesis)
    #     # print(wer)
    #     wer_scores.append(wer)
    # wer_mean=sum(wer_scores) / len(wer_scores)
    
    return reference,hypothesis

def get_detail_wer(gold_list,test_list):
    out = jiwer.process_words(
            gold_list,
                test_list,
            )
    print("wer: ",out.wer*100)
    print("hits: ",out.hits)
    print("substitutions: ",out.substitutions)
    print("insertions: ",out.insertions)
    print("deletions: ",out.deletions)
    print("total words: ",out.hits+out.substitutions+out.deletions)

# def get_t_p_wer(substitutions, insertions,deletions,total_words_punc, P_WER):
#     t_p_wer=(substitutions+insertions+deletions)/total_words_punc
#     print("Topline of P-WER is ",t_p_wer*100)
#     print("Relative PER is ", P_WER-t_p_wer*100)

def get_c_p_er_fr_wer(wer,C_WER,P_WER,clean=True):
    if clean:
        total_words=50082
        total_words_punt=57312
        cased_words=3849
        p_num = 9709
    else:
        total_words=48488
        total_words_punt=55896
        cased_words=4231
        p_num = 9408
    error_words = wer*total_words
    error_c = C_WER*total_words
    cer=(error_c-error_words)/cased_words
    error_p = P_WER*total_words_punt
    per=(error_p-error_words)/p_num
    # print("Topline of P-WER is ",t_p_wer)
    print("Relative CER is ", cer)
    print("Relative PER is ", per)

def get_c_p_er_fr_wer_ami(wer,C_WER,P_WER):
    # if clean:
    total_words=94006
    total_words_punt=110266
    cased_words=7626
    p_num = 16260
    error_words = wer*total_words
    error_c = C_WER*total_words
    cer=(error_c-error_words)/cased_words
    error_p = P_WER*total_words_punt
    per=(error_p-error_words)/p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)

def get_cased_word(text_list):
    capitalized_word_count = 0

    # Iterate through the list of text
    for text in text_list:
        # Split the text into words
        words = text.split()
        
        # Check each word in the text
        for word in words:
            # Check if the word is capitalized (first character is uppercase)
            if word.isalpha() and word[0].isupper():
                capitalized_word_count += 1
    print("Cased words number is ", capitalized_word_count)

In [3]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/POC/dictation/books-transcription/test-clean-rich-book.txt","clean.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

ValueError: not enough values to unpack (expected 2, got 1)

In [7]:
get_c_p_er_fr_wer(wer=2.2674650698602794,C_WER=7.9128070417991605,P_WER=9.537098405832491,clean=True)

Relative CER is  80.90103503048464
Relative PER is  44.60098858856002


In [24]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-other-rich-book.txt","other.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
print("WER")
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2731
CP-WER
wer:  18.42241733874573
hits:  46071
substitutions:  6713
insertions:  452
deletions:  3137
total words:  55921
P-WER
wer:  12.598129504121886
hits:  49324
substitutions:  3464
insertions:  448
deletions:  3133
total words:  55921
C-WER
wer:  12.035376337435833
hits:  42902
substitutions:  5335
insertions:  233
deletions:  270
total words:  48507
WER
wer:  5.2404807553548975
hits:  46197
substitutions:  2041
insertions:  232
deletions:  269
total words:  48507


In [22]:
get_c_p_er_fr_wer(wer=5.2404807553548975,C_WER=11.69739874876523,P_WER=12.598129504121886,clean=False)

Relative CER is  73.99740951653988
Relative PER is  47.84062668970543


In [28]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/vasr/calculate_wer/ami_test_gold_process.txt","ihm.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

9474
CP-WER
wer:  45.245134492953405
hits:  63499
substitutions:  26188
insertions:  3123
deletions:  20579
total words:  110266
P-WER
wer:  42.38296483050079
hits:  66730
substitutions:  22882
insertions:  3198
deletions:  20654
total words:  110266
C-WER
wer:  42.23772950662724
hits:  56542
substitutions:  22490
insertions:  2242
deletions:  14974
total words:  94006
wer:  38.777312086462565
hits:  59844
substitutions:  19139
insertions:  2291
deletions:  15023
total words:  94006


In [5]:
get_c_p_er_fr_wer_ami(wer=38.777312086462565,C_WER=42.23772950662724,P_WER=42.38296483050079)

Relative CER is  36.62470495672692
Relative PER is  95.6519065190652


In [15]:
# Define the input and output file paths
input_file_path = 'ihm.txt'
output_file_path = 'ihm_processed.txt'

# Open the input and output files
with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
    # Iterate through each line in the input file
    for line in input_file:
        # Split the line into ID and text
        parts = line.split(',')
        if len(parts) == 2:
            id = parts[0]
            text = parts[1].strip()
            
            # Capitalize the first word of the text
            text = text.capitalize()
            
            # Write the processed line to the output file
            output_file.write(f"{id},{text}\n")

print("File processed and saved to", output_file_path)


File processed and saved to ihm_processed.txt
